# Análisis de Tweets #

<div align="justify">
Variables para TweetPy
</div>

| Variable        | Tipo de dato  | Descripción                       |
| --------------- | ------------: | --------------------------------- |
| consumer_key    | String        | Consumidor llave                  |
| consumer_secret | String        | Consumidor secreto                |
| access_key      | String        | Acceso llave                      |
| access_secret   | String        | Acceso secreto                    |

In [1]:
consumer_key = "gBlPQ75wdroVezt5HoYIvErzm"
consumer_secret = "LJNbWEqePHa7GiNpCRKKSS01FoIOZV7Wdfmnxo9V1GHlHXOVrM"
access_key = "784264322893557761-SkRdYJjnWUiqbauxRaBBTvMNGA4yP8i"
access_secret = "BC9p8mnkyTsIqXH7ydYtzbs3QYJ6lvoasy7okjsXgYEdO"

<div align="justify">
Variables para OandaPy
</div>

| Variable     | Tipo de dato  | Descripción                       |
| :----------: | ------------: | --------------------------------- |
| OA_AT        | String        | Tipo de cuenta                    |
| OA_Gn        | String        | Frecuencia de muestra de precio   |
| OA_In        | String        | Instrumento Financiero a utilizar |
| OA_Ai        | Entero        | ID de cuenta                      |
| OA_Ta        | String        | Uso horario (No aplica)           |

In [2]:
OA_At = "practice"           # Tipo de cuenta.
OA_Gn = "M5"                 # Frecuencia de muestra de precio.
OA_In = "USD_MXN"            # Instrumento Financiero a utilizar.
OA_Ai = 1742531              # ID de cuenta
OA_Ta = "America/Monterrey"  # Uso horario
OA_Tk = "ada4a61b0d5bc0e5939365e01450b614-4121f84f01ad78942c46fc3ac777baa6"

<div align="justify">
Declaramos nuesto usuario a consultar y una cantidad de tweets a consultar desde el último publicado.
</div>

In [3]:
usuario_a_consultar = 'realDonaldTrump'
cantidad_tweets = 1000

<div align="justify">
Se guarda en una lista las palabras a consultar.
</div>

In [4]:
lista_palabras = ['mexico', 'illega', 'nafta', 'immigra']

<div align="justify">
Se crea una función para poder consultar información de tweeter y guarda el resultado en una variable. <br> Para ello también se debe importar Tweepy.
</div>

In [5]:
import tweepy

In [6]:
def retornar_api(consumer_key, consumer_secret, access_key, access_secret):
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api_retorno = tweepy.API(auth)
    return api_retorno

In [7]:
api = retornar_api(consumer_key, consumer_secret, access_key, access_secret)

<div align="justify">
Para un mejor tratamiento y manimulacion de datos, se importa la librería pandas para esto.
</div>

In [8]:
import pandas as pd

<div align="justify">
La siguiente función regresa un data frame de pandas, mandando como argumentos, la variable que regresa tweepy, el nombre de usuario y la cantidad de tweets a consultar.<br><br>
Se agregó una variable llamada irregular la cual mide si la api no nos regresa exactamente 200 tweets y hubo un ciclo donde no fue 200 el retorno de tweets, es por eso que el ultimo tweet es 998, es decir, nuestro conjuntos de datos tiene un tamaño de 999 tweets.
</div>

In [9]:
def consultar_tweets_usuario(tweeter_api, usuario, cantidad):
    indice = 0
    irregular = 0
    tweets = tweeter_api.user_timeline(screen_name=usuario, count=200)
    ultimo_tweet = tweets[-1].id - 1
    columnas = ('texto', 't_inicial', 'retweets', 'favoritos')
    df = pd.DataFrame(columns=columnas)
    for t in tweets:
        df.loc[indice] = t.text, t.created_at, t.retweet_count, t.favorite_count
        indice += 1
    while len(tweets) > 0 and df.shape[0] + irregular < cantidad:
        tweets = tweeter_api.user_timeline(screen_name=usuario, count=200, max_id=ultimo_tweet)
        irregular = irregular + (200 - len(tweets))
        for t in tweets:
            df.loc[indice] = t.text, t.created_at, t.retweet_count, t.favorite_count
            indice += 1
        ultimo_tweet = tweets[-1].id - 1
    return df

<div align="justify">
Para vizualizar el conjuntos de datos, se crea una función, regresando el mismo conjuntos de datos con un estilo definido por pandas.
</div>

In [ ]:
def formatear_tabla(df):
    return df.style

<div align="justify">
Se visualiza el encabezado del conjunto de datos
</div>

In [ ]:
datos = consultar_tweets_usuario(api, usuario_a_consultar, cantidad_tweets)
formatear_tabla(datos.head())

<div align="justify">
También se visualiza los ultimos tweets para verificar la cantidad.
</div>

In [ ]:
formatear_tabla(datos.tail())

In [ ]:
def cantidad_tweets(df):
    return df.shape[0]

In [ ]:
cantidad_tweets(datos)

<div align="justify">
Utilizando la lista de palabras para encontrar, se busca si existe una o más palabras en cada tweet, para guardar
los tweets que se interesan y que palabras están en cada uno.
</div>

In [ ]:
import re

In [ ]:
def palabras_en_texto(palabras, texto):
    texto = re.sub(r'\W', '', texto).lower()
    detectado = False
    lista = list()
    for palabra in palabras:
        palabra = palabra.lower()
        relacion = re.search(palabra, texto)
        if relacion:
            detectado = True
            lista.append(palabra)
    return detectado, '-'.join(lista)

In [ ]:
def filtrar_por_palabras(df, palabras):
    retorno = df['texto'].apply(lambda tweet: palabras_en_texto(palabras, tweet))
    df['encontrado'] = tuple(map(lambda x: x[0], retorno))
    df['palabras'] = tuple(map(lambda x: x[1], retorno))
    df = df[df.encontrado]
    df = df.drop(['encontrado'], axis=1)
    return df

<div align="justify">
Se puede observar que nuestro conjunto de datos es más corto, y una nueva columna que guarda que palabras encontró en cada 
tweet.
</div>

In [ ]:
datos = filtrar_por_palabras(datos, lista_palabras)
cantidad_tweets(datos)

<div align="justify">
Ahora solo se tiene 35 tweets de los 1000 que se obtuvieron.
</div>

In [ ]:
formatear_tabla(datos.head())

<div align="justify">
Tambien se debe excluir los tweets en fechas donde no hay precios.
</div>

In [ ]:
def filtrar_tiempos(df):
    df['t_inicial'] = pd.to_datetime(df['t_inicial'])
    # Remover los sabados
    sabado = df[df['t_inicial'].dt.dayofweek == 5]
    # Remover los viernes despues de las 22:00:00 utc-0
    viernes = df[df['t_inicial'].dt.dayofweek == 4]
    viernes = viernes[viernes['t_inicial'].dt.hour > 22]
    # Remover los domingos antes de las 12:00:00 utc-0
    domingo = df[df['t_inicial'].dt.dayofweek == 6]
    domingo = domingo[domingo['t_inicial'].dt.hour < 22]
    df = df.drop(sabado.index)
    df = df.drop(viernes.index)
    df = df.drop(domingo.index)
    return df

In [ ]:
datos = filtrar_tiempos(datos)
cantidad_tweets(datos)

In [ ]:
formatear_tabla(datos.head())

<div align="justify">
Una vez filtrados los tweets por palabras y las fechas donde hay precios, se procede a extraer los precios de Oanda,
para utilizar la libreria Oandapy, primero se le da un formato valido a cada fecha.
En este caso de tuvo que unir la fecha y el tiempo con una T.
<br>Ejemplo:
<br><br>Antes: 2017-03-03 02:38:48
<br>Después: 2017-03-03T02:38:48
</div>

In [ ]:
def formato_tiempo(df):
    df['t_inicial'] = df.t_inicial.apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%S'))
    return df

<div align="justify">
Validando el formato deseado
</div>

In [ ]:
datos = formato_tiempo(datos)
formatear_tabla(datos.head())

<div align="justify">
En la función obtener_precios se manda el conjunto de datos, con los parametros que se inicializarón al principio.
Se obtienen 3 nuevas columnas en nuestro conjunto de datos.
<br><br>
precio_inicial = precio en el minuto de la fecha que fue publicado el tweet.
<br><br>
precio_final = precio hora después de la fecha que fue publicado el tweet.
<br><br>
volatilidad = Se obtiene a partir de los rendimientos de cada 5 minutos, en el lapso de tiempo desde que se publicó el tweet hasta una hora después, despues de obtener los rendimientos, se calcula la desviación estandar y se multiplica por 100.
</div>

In [ ]:
import oandapy
import numpy as np

In [ ]:
def obtener_precios(df, oa_at, oa_tk, oa_in, oa_ta,
                    oa_gn):
    oanda = oandapy.API(environment=oa_at, access_token=oa_tk)
    p_inicial = list()
    p_final = list()
    volatilidad = list()
    for indice, fila in df.iterrows():
        precios = oanda.get_history(instrument=oa_in, start=fila['t_inicial'], dailyAlignment=1,
                                    alignmentTimezone=oa_ta, granularity=oa_gn, count=12)
        precios = pd.DataFrame(precios['candles'])
        p_inicial.append(precios['closeAsk'][0])
        p_final.append(precios['closeAsk'][1])
        rendimientos = np.log(precios['closeAsk'] / precios['closeAsk'].shift(1))
        volatilidad.append(np.std(rendimientos) * 100)
    df['p_inicial'] = p_inicial
    df['p_final'] = p_final
    df['volatilidad'] = volatilidad
    return df

In [ ]:
datos = obtener_precios(datos, OA_At, OA_Tk, OA_In, OA_Ta, OA_Gn)
formatear_tabla(datos.head())

<div align="justify">
Se marca tendencia para cada tweets para saber si subio o bajo el precio una hora despues de que fue publicado.
</div>

In [ ]:
def marcar_tendencia(df):
    df['tendencia'] = ""
    df.loc[(df['p_inicial'] > df['p_final']), 'tendencia'] = 'bajista'
    df.loc[(df['p_inicial'] < df['p_final']), 'tendencia'] = 'alcista'
    df.loc[(df['p_inicial'] == df['p_final']), 'tendencia'] = 'igual'
    return df

In [ ]:
datos = marcar_tendencia(datos)
formatear_tabla(datos.head())

<div align="justify">
Se calcula el rendimiento en pips, se calcula de 2 maneras diferentes dependiendo de su tendencia.
<br><br>
alcista = (precio_inicial - precio_final) x 10000
<br><br>
bajista = (precio_final - precio_inicial) x 10000
</div>

In [ ]:
def rendimientos_en_pips(df):
    df['rendimiento'] = 0.0
    df.loc[(df['tendencia'] == 'bajista'), 
           'rendimiento'] = (df['p_inicial'] - df['p_final']) * 10000
    
    df.loc[(df['tendencia'] == 'alcista'),
           'rendimiento'] = (df['p_final'] - df['p_inicial']) * 10000
    return df

In [ ]:
datos = rendimientos_en_pips(datos)
formatear_tabla(datos.head())